## Under pressure

In [295]:
from pathlib import Path
import numpy as np
import re
import itertools

In [296]:
pattern = 'Valve (.+) has flow rate=(\d+); tunnels? leads? to valves? (.+)'
pattern = re.compile(pattern)

In [319]:
valves = Path('valves.txt').read_text().split('\n')

valve_map = {}

for line in valves:
    [key, flow, stations] = re.match(pattern, line).groups()
    
    valve_map[key] = [stations.split(', '), int(flow)]

# Identify valves with non-zero flow and order them from largest to lowest flow
non_zero_valves = [key for key in valve_map if valve_map[key][1] > 0]
non_zero_valves = sorted(non_zero_valves, key = lambda x: valve_map[x][1], reverse = True)

In [320]:
def find_shortest_path(start, end):
    paths = [[start]]
    visited = [start]
    
    if start == end:
        return [start]
    
    while end not in visited:
        new_paths = []
        
        for path in paths:
            last = path[-1]
            routes = valve_map[last][0]
            
            
            for route in routes:
                if route in path:
                    continue
                
                else:
                    if route not in visited:
                        visited += [route]
                    
                    new_path = path + [route]
                
                    new_paths += [new_path]
                    
                    if route == end:
                        return new_path
        
        paths = new_paths
                
    return None

In [321]:
def find_total_path(locations):
    path = []
    
    for i in range(1, len(locations)):
        route = find_shortest_path(locations[i-1], locations[i])
        
        if i == 1:
            path += route
        else:
            path += route[1:]
        
    return path

In [322]:
def calculate_path_time(path):
    if len(path) == 0:
        return 0
    else:
        if path[-1] == 'STOP':
            path = path[0:-1]
        time = 0
        for i in range(1, len(path)):
            time += distances[path[i-1], path[i]]
            
        return time

In [323]:
def calculate_total_pressure(path, time_limit = 30):
    if path[-1] == 'STOP':
        path = path[0:-1]
        
    times = np.zeros(len(path), dtype = 'int')
    for i in range(1, len(path)):
        times[i] = times[i-1] + distances[path[i-1], path[i]] + 1
        
    pressures = np.array([valve_map[valve][1] for valve in path])
    
    time_remaining = time_limit - times
    
    time_remaining[time_remaining < 0] = 0
    
    pressure_released = np.sum(time_remaining * pressures)
        
    return pressure_released
        

In [324]:
class decision():
    
    def __init__(self, location, valves = [], pressures = [], total_pressure = 0, time = 0, prev_decision = None):
        
        self.location = location
        
        self.valves_open = valves
        self.pressures_open = pressures
        self.total_pressure_released = total_pressure
        self.total_pressure_released += np.sum(pressures)
        
        self.time = time
        
        self.prev_decision = prev_decision
        
    def __repr__(self):
        if self.prev_decision == None:
            return f'Start at {self.location}'
        
        elif len(self.prev_decision.valves_open) == len(non_zero_valves):
            return 'Do nothing'
            
        elif self.location != self.prev_decision.location:
            return f'Go to {self.location} from {self.prev_decision.location}'
        
        elif self.location == self.prev_decision.location and self.location in self.valves_open:
            return f'Open valve at {self.location}'
        
        
    
    def __str__(self):
        return self.__repr__()
    
    def __eq__(self, other):
        if not isinstance(other, decision):
            return False
        else:
            return (self.location == other.location and self.valves_open == other.valves_open and self.pressures_open == other.pressures_open 
                    and self.total_pressure_released == other.total_pressure_released and self.time == other.time)
            
    
    def make_decisions(self):
        
        valve_open = self.location in self.valves_open
        
        stations, flow = valve_map[self.location]
        
        possible_decisions = []
        
        remaining_valves = np.setdiff1d(non_zero_valves, self.valves_open)
        
        if len(remaining_valves) == 0: # All valves open, do nothing
            return [decision(self.location, self.valves_open, self.pressures_open, self.total_pressure_released, self.time+1, self)]
        
        # Consider opening valve
        elif not valve_open and flow > 0:
            new_valves_open = self.valves_open + [self.location]
            new_pressures_open = self.pressures_open + [flow]
            
            # If this valve is the highest priority remaining valve, just open it:
            if remaining_valves[0] == self.location:
                return [decision(self.location, valves = new_valves_open, 
                                           pressures = new_pressures_open, 
                                           total_pressure = self.total_pressure_released,
                                           time = self.time+1, prev_decision = self)]
            
            else:
                possible_decisions += [decision(self.location, valves = new_valves_open, 
                                           pressures = new_pressures_open, 
                                           total_pressure = self.total_pressure_released,
                                           time = self.time+1, prev_decision = self)]
            
        # Path towards closed valves
        else:
            destinations = [find_shortest_path(self.location, valve)[1] for valve in remaining_valves if valve != self.location]
            destinations = set(destinations)
                            
            possible_decisions += [decision(dest, self.valves_open, self.pressures_open, self.total_pressure_released, self.time+1, self) for dest in destinations]
            
        return possible_decisions

In [325]:
locations = ['AA'] + non_zero_valves

distances = {}

for start in locations:
    for end in locations:
        distances[start, end] = len(find_shortest_path(start, end))-1

In [326]:
start = 'AA'

num_nzv = len(non_zero_valves)

paths = [[start]]

time_limit = 30

for i in range(num_nzv):
    new_paths = []
    
    for path in paths:
        if path[-1] == 'STOP':
            continue
        
        elif calculate_path_time(path) > time_limit - len(path):
            new_paths += [path + ['STOP']]
            continue
        
        remaining_valves = np.setdiff1d(non_zero_valves, path)
        
        for valve in remaining_valves:
            new_path = path + [valve]
            
            new_paths += [new_path]
    
    if len(new_paths) == 0:
        break
        
    paths = new_paths

In [327]:
max([calculate_total_pressure(path) for path in paths])

1673

## Part 2: This is our last dance

In [328]:
start = 'AA'

num_nzv = len(non_zero_valves)

paths = [[start]]

time_limit = 26

for i in range(num_nzv):
    new_paths = []
    
    for path in paths:
        if path[-1] == 'STOP':
            continue
        
        elif calculate_path_time(path) > time_limit - len(path):
            new_paths += [path + ['STOP']]
            continue
        
        remaining_valves = np.setdiff1d(non_zero_valves, path)
        
        for valve in remaining_valves:
            new_path = path + [valve]
            
            new_paths += [new_path]
    
    if len(new_paths) == 0:
        break
        
    paths = new_paths

In [329]:
paths

[['AA', 'GB', 'GR', 'CQ', 'GV', 'HX', 'JI', 'XM', 'BX', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'GV', 'HX', 'JI', 'XM', 'HF', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'GV', 'HX', 'JI', 'XM', 'IR', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'GV', 'HX', 'JI', 'XM', 'LC', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'GV', 'HX', 'JI', 'XM', 'OH', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'GV', 'HX', 'JI', 'XM', 'OK', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'GV', 'HX', 'JI', 'XM', 'TS', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'GV', 'HX', 'JI', 'XM', 'UN', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'HF', 'GV', 'HX', 'IR', 'BX', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'HF', 'GV', 'HX', 'IR', 'JI', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'HF', 'GV', 'HX', 'IR', 'LC', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'HF', 'GV', 'HX', 'IR', 'OH', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'HF', 'GV', 'HX', 'IR', 'OK', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'HF', 'GV', 'HX', 'IR', 'TS', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ', 'HF', 'GV', 'HX', 'IR', 'UN', 'STOP'],
 ['AA', 'GB', 'GR', 'CQ',

In [337]:
def calculate_two_path_pressure(path1, path2, time_limit = 30):
    if path1[-1] == 'STOP':
        path1 = path1[0:-1]
    if path2[-1] == 'STOP':
        path2 = path2[0:-1]
        
    time1 = np.zeros(len(path1), dtype = 'int')
    time2 = np.zeros(len(path2), dtype = 'int')
    
    for i in range(1, len(path1)):
        time1[i] = time1[i-1] + distances[path1[i-1], path1[i]] + 1
        
    for i in range(1, len(path2)):
        time2[i] = time2[i-1] + distances[path2[i-1], path2[i]] + 1
        
    time1 = {station: time_limit - time for station, time in zip(path1, time1)}
    time2 = {station: time_limit - time for station, time in zip(path2, time2)}
    
    all_keys = set(list(time1.keys()) + list(time2.keys()))
    best_times = {}
    
    for key in all_keys:
        if key in time1 and key in time2:
            best_times[key] = max([time1[key], time2[key], 0])
        elif key in time1 and key not in time2:
            best_times[key] = max(time1[key], 0)
        elif key not in time1 and key in time2:
            best_times[key] = max(time2[key], 0)
            
    pressure_lost = sum([best_times[valve] * valve_map[valve][1] for valve in best_times])
    
    return pressure_lost, time1, time2, best_times
    

In [339]:
best_path_pressure = max([calculate_total_pressure(path, time_limit = 26) for path in paths])
best_one_person_paths = [path for path in paths if calculate_total_pressure(path) == best_path_pressure]

In [340]:
best_one_person_paths

[['AA', 'LC', 'HF', 'OK', 'CQ', 'GR', 'JI', 'XM', 'BX', 'STOP'],
 ['AA', 'LC', 'OK', 'CQ', 'GR', 'JI', 'XM', 'OH', 'BX', 'STOP'],
 ['AA', 'LC', 'OK', 'HF', 'CQ', 'GR', 'GB', 'XM', 'OH', 'STOP'],
 ['AA', 'LC', 'UN', 'GV', 'HX', 'JI', 'GR', 'CQ', 'BX', 'STOP'],
 ['AA', 'LC', 'UN', 'GV', 'HX', 'JI', 'GR', 'CQ', 'GB', 'STOP'],
 ['AA', 'LC', 'UN', 'GV', 'HX', 'JI', 'GR', 'CQ', 'IR', 'STOP'],
 ['AA', 'LC', 'UN', 'GV', 'HX', 'JI', 'GR', 'CQ', 'OH', 'STOP'],
 ['AA', 'LC', 'UN', 'GV', 'HX', 'JI', 'GR', 'CQ', 'TS', 'STOP'],
 ['AA', 'LC', 'UN', 'GV', 'HX', 'JI', 'GR', 'CQ', 'XM', 'STOP'],
 ['AA', 'UN', 'OK', 'HF', 'CQ', 'GR', 'JI', 'HX', 'IR', 'STOP']]

In [347]:
for path in best_one_person_paths:
    start = 'AA'
    
    remaining_valves = np.setdiff1d(non_zero_valves, path)

    num_nzv = len(remaining_valves)

    comp_paths = [[start]]

    time_limit = 26

    for i in range(num_nzv):
        new_paths = []

        for path2 in comp_paths:
            if path2[-1] == 'STOP':
                continue

            elif calculate_path_time(path2) > time_limit - len(path2):
                new_paths += [path2 + ['STOP']]
                continue

            remaining_valves2 = np.setdiff1d(remaining_valves, path2)

            for valve in remaining_valves:
                new_path = path2 + [valve]

                new_paths += [new_path]

        if len(new_paths) == 0:
            break

        comp_paths = new_paths

In [348]:
comp_paths

[['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'BX', 'BX'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'BX', 'GB'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'BX', 'GV'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'BX', 'LC'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'BX', 'OH'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'BX', 'TS'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'BX', 'XM'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GB', 'BX'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GB', 'GB'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GB', 'GV'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GB', 'LC'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GB', 'OH'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GB', 'TS'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GB', 'XM'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GV', 'BX'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GV', 'GB'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GV', 'GV'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GV', 'LC'],
 ['AA', 'BX', 'BX', 'BX', 'BX', 'BX', 'GV', 'OH'],
 ['AA', 'BX', 'BX', 'BX', 'BX',

In [346]:
remaining_valves

array(['BX', 'GB', 'GV', 'LC', 'OH', 'TS', 'XM'], dtype='<U2')

In [275]:
paths[0]

['AA', 'GB', 'GR', 'CQ', 'GV', 'HX', 'JI', 'XM', 'BX', 'STOP']

In [278]:
[element in paths[0] for element in ['GB', 'GR']]

[True, True]

In [ ]:
[path2 for path2 in paths if np.intersect1d(path[0][1:], path2[1:]) == []]

In [ ]:
best_one_person

In [198]:
start = 'AA'

num_nzv = len(non_zero_valves)

paths = [[[start], [start]]]

time_limit = 26

for i in range(15):
    print(i)
    new_paths = []
    
    for my_path, elephant_path in paths:
        
        remaining_valves = np.setdiff1d(non_zero_valves, my_path + elephant_path)
        
        my_new_path = my_path
        elephant_new_path = elephant_path
        
        if len(remaining_valves) == 0:
            my_new_path += ['STOP']
            elephant_new_path += ['STOP']
        
        if my_path[-1] == 'STOP' and elephant_path[-1] == 'STOP':
            continue
        
        if calculate_path_time(my_path) > time_limit - len(my_path):
            my_new_path += ['STOP']
        
        elif calculate_path_time(elephant_path) > time_limit - len(elephant_path):
            elephant_new_path += ['STOP']
        
        if my_new_path[-1] == 'STOP' and elephant_new_path[-1] != 'STOP':
            for valve in remaining_valves:
                elephant_new_path = elephant_path + [valve]
                new_paths += [[my_new_path, elephant_new_path]]
        
        elif my_new_path[-1] == 'STOP' and elephant_new_path[-1] == 'STOP':
            for valve in remaining_valves:
                my_new_path = my_path + [valve]
                new_paths += [[my_new_path, elephant_new_path]]
                
        elif my_new_path[-1] == 'STOP' and elephant_new_path[-1] == 'STOP':
            new_paths += [[my_new_path, elephant_new_path]]
            
        else:
        
            for my_valve in remaining_valves:
                for elephant_valve in np.setdiff1d(remaining_valves, my_valve):
                    my_new_path = my_path + [my_valve]
                    elephant_new_path = elephant_path + [elephant_valve]

                    new_paths += [[my_new_path, elephant_new_path]]
    
    if len(new_paths) == 0:
        break
        
    paths = new_paths

0
1
2
3


KeyboardInterrupt: 

In [199]:
len(paths)

3603600